# Model Inference Code

<b> Table of Contents</b>
- Imports
- Loading model into memory
- Inference on all the images and summary generation

## Checking the installation of Tensorflow Object detection API

In [1]:
!ls

area_report_after_inference_08052020.csv
confidence_report_after_inference_08052020.csv
count_report_after_inference_08052020.csv
eval_val_exp1_iter8
fine_tuned_model_exp1_iter3_25000
fine_tuned_model_iter_15000
frozen_inference_graph_18895_cutoff0.5
frozen_inference_graph_18895.pb
frozen_inference_graph_20000_cutoff0.5
frozen_inference_graph_20000.pb
frozen_inference_graph_iter1_step13871.pb
frozen_inference_graph_iter1_step17815.pb
frozen_inference_graph_iter2_step19714.pb
frozen_inference_graph_iter3_step23740.pb
frozen_inference_graph_iter3_step25000.pb
frozen_inference_graph_iter3_step27829.pb
frozen_inference_graph_iter4_step20812.pb
frozen_inference_graph_iter5_step28000.pb
frozen_inference_graph_iter8_step30000.pb
frozen_inference_graph.pb
home
iter3_step25000
iter4_step20812
iter5_step28000
iter7_20000
iter8_step30000
iter9_step15000
known
Object_Detection_Inference_SSD_v5.1.2-Copy1.ipynb
Object_Detection_Inference_SSD_v5.1.2.ipynb
Object_Detection_Inference_SSD_v5.1.3.ipynb
o

In [2]:
%cd /home/ubuntu/models/research

/home/ubuntu/models/research


In [3]:
!echo $PYTHONPATH

:/home/ubuntu/models/research:/home/ubuntu/models/research/slim


In [4]:
!python object_detection/builders/model_builder_test.py

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Running tests under Python 3.6.10: /home/ubuntu/.conda/envs/dristi2/bin/python
[ RUN      ] ModelBuilderTest.test_create_experimental_model
[       OK ] ModelBuilderTest.test_create_experimental_model
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_model_from_config_with_example_miner
[       OK ] ModelBuilderTest.test_create_faster_rcnn_model_from_config_with_example_miner
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_with_matmul
[       OK ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_with_matmul
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_model

In [5]:
#------main part of code

## Imports

In [6]:
# importing library
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
import time
import csv
import pandas as pd
import pathlib
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import math

Using TensorFlow backend.


## Environment Setup

In [7]:
# This is needed to display the images.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [8]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

Patches:

In [9]:
if tf.__version__.startswith('2.'):
  # patch tf1 into `utils.ops`
  utils_ops.tf = tf.compat.v1

  # Patch the location of gfile
  tf.gfile = tf.io.gfile

## Configurations and file paths

In [10]:
# Mention the name of the model .pb and the labels .pbtxt file
# Path to frozen detection graph. This is the actual model that is used for the object detection.
frozen_model_fname = '/home/ubuntu/dristi_ssd_model/frozen_inference_graph_20000.pb'

# List of the strings that is used to add correct label for each box.
label_map_pbtxt_fname = '/home/ubuntu/dristi_ssd_model/object-detection.pbtxt' # labels file path

# Number of classes
NUM_CLASSES = 5

# Path to the test images folder for inference - set the zip folder path if zip folder is uploaded
test_img_folder_path = '/home/ubuntu/dristi_ssd_model/test_combined'

# Size, in inches, of the output images.
IMAGE_SIZE = (24, 24)

# Path to the binary classifiaction model file
model_path = "/home/ubuntu/dristi/resnet50_Png_full_data_acc_split.model"

# Threshold for binary classifcation
threshold = 0.5

# Threshold for object detection
cutoff = 0.4
# cutoff = 1e-8

# IoU threshold for non-max suppression
iou_threshold=0.1

# List of defect - in the order of annotation
defect_cols = ['Mistracking','Trim','Contamination','Tear','Wrinkle/Fold']

# Image size for binary classification
img_rows, img_cols = 512,512

## Model preparation
### Variables
Any model exported using the export_inference_graph.py tool can be loaded here

In [11]:
# # Extracting the tar model file - incase we upload a tar model folder - run when upload a new tar folder
# tar = tarfile.open("/home/ubuntu/dristi/fine_tuned_modelv_with_dist_color_finetuned19713.tar.gz")
# tar.extractall()
# tar.close()

In [12]:
# Checking the folder contents - QC step (whether the frozen graph is present or not)
# !ls fine_tuned_modelv_with_dist_color_finetuned19713

In [13]:
# Displaying the present worrking directory - should be research folder
!pwd

/home/ubuntu/models/research



### Load a (frozen) Tensorflow model into memory

In [14]:
%%time
def load_model(frozen_model_fname):
    detection_graph = tf.Graph()
    with detection_graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(frozen_model_fname, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')
    return detection_graph
detection_graph = load_model(frozen_model_fname)

CPU times: user 528 ms, sys: 176 ms, total: 704 ms
Wall time: 705 ms


### Loading the binary classification model into memory

In [15]:
#Initializing own performance metric

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [16]:
# Loading the binary classification model into memory
from keras.models import load_model
st = time.time()
model = load_model(model_path, custom_objects={'f1': f1})
print("time taken", time.time()-st)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

time taken 18.157276153564453


### Loading label map
Label maps map indices to category names, so that when our convolution network predicts 1, we know that this corresponds to Mistracking and when it predicts 2, we know it corresponds to Trim, and so on. Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine.

In [17]:
label_map = label_map_util.load_labelmap(label_map_pbtxt_fname)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

### Helper code

In [18]:
# Since we need our images in a specific format - since 3 channel image is accepted
def load_image_into_numpy_array(image):
    # The function supports only grayscale images
    last_axis = -1
    dim_to_repeat = 2
    repeats = 3
    grscale_img_3dims = np.expand_dims(image, last_axis)
    training_image = np.repeat(grscale_img_3dims, repeats, dim_to_repeat).astype('uint8')
    assert len(training_image.shape) == 3
    assert training_image.shape[-1] == 3
    return training_image

In [19]:
# #use this function when you have ground truth boxes
# def load_image_into_numpy_array(image):
#     (im_width, im_height) = image.size
#     return np.array(image.getdata()).reshape(
#         (im_height, im_width, 3)).astype(np.uint8)

## Detection in images
### Reading the various files and folders

In [20]:
# Unzip the folder - uncomment when uploading new images in zip folder
#  !unzip {test_img_folder_path}
#  Setting the foldername now as the path as the folder gets unzipped in the pwd
#  test_img_folder_path = test_img_folder_path.split('/')[-1].split('.')[0]

In [21]:
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = pathlib.Path(test_img_folder_path)
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

[PosixPath('/home/ubuntu/dristi_ssd_model/test_combined/VISUCI1_S1_20191207_050605_819_3.jpg'),
 PosixPath('/home/ubuntu/dristi_ssd_model/test_combined/VISUCI1_S1_20191208_123531_849_3.jpg'),
 PosixPath('/home/ubuntu/dristi_ssd_model/test_combined/VISUCI1_S1_20191208_172306_479_1.jpg'),
 PosixPath('/home/ubuntu/dristi_ssd_model/test_combined/VISUCI1_S1_20191210_174100_732_3.jpg'),
 PosixPath('/home/ubuntu/dristi_ssd_model/test_combined/VISUCI1_S1_20191210_211959_279_3.jpg'),
 PosixPath('/home/ubuntu/dristi_ssd_model/test_combined/VISUCI1_S1_20191211_132955_017_3.jpg'),
 PosixPath('/home/ubuntu/dristi_ssd_model/test_combined/VISUCI1_S1_20191211_143347_920_3.jpg'),
 PosixPath('/home/ubuntu/dristi_ssd_model/test_combined/VISUCI1_S1_20191213_101200_470_3.jpg'),
 PosixPath('/home/ubuntu/dristi_ssd_model/test_combined/VISUCI1_S1_20191216_182320_254_3.jpg'),
 PosixPath('/home/ubuntu/dristi_ssd_model/test_combined/VISUCI1_S1_20191217_041555_632_3.jpg'),
 PosixPath('/home/ubuntu/dristi_ssd_mode

## This section is used to get the inference on the images
- First, it classifies the images into good or bad
- Then, for the bad images only we do the inference
- The defect is identified as well as localised
- In case, there are no known defects identified in the defected images, we mark it as an unknown defect
- In case of good(no defect) images it is just marked as good

### Binary classification inference

In [22]:
# This function takes in an image and infers it to have defect/no defect(good)
def binary_classification_inference(threshold,TEST_IMAGE_PATHS,model):
    from keras.preprocessing import image
    start = time.time()   
    def add_prediction(image_path):
        st_bc = time.time()
        image_key = str(image_path).split('/')[-1].split('.')[0]
        img = image.load_img(image_path, target_size = (img_rows, img_cols))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis = 0)
        img /= 255
        prob = model.predict(img)[0][0]
        print("Time taken for binary classification inference of 1 image",time.time() - st_bc)
        return (image_key,prob,int(prob<threshold))
    predictions = [add_prediction(image_path) for image_path in TEST_IMAGE_PATHS]
    predicted_label_df = pd.DataFrame(predictions,columns = ['Image_Name','Probability','Predicted_Class'])
    print(predicted_label_df)
    end = time.time()
    print("Time taken for binary classification inference of all images",end - start)
    return predicted_label_df.set_index("Image_Name")

### Object detection inference - including binary classifaction

In [23]:
%%time
%matplotlib inline
def get_inference(predicted_label_df,TEST_IMAGE_PATHS,cutoff):
    from collections import defaultdict
    start = time.time()
    with detection_graph.as_default():
        with tf.Session(graph=detection_graph) as sess:
            # Definite input and output Tensors for detection_graph
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            # Each box represents a part of the image where a particular object was detected.
            detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            # Each score represent how level of confidence for each of the objects.
            # Score is shown on the result image, together with the class label.
            detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
            detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            image_summary = pd.DataFrame(columns=['Image_key']+defect_cols+['Unknown','Good'])
            
            total_count = 0
            for image_path in TEST_IMAGE_PATHS:
                print(image_path)
                st_od = time.time()
                image_key = str(image_path).split('/')[-1].split('.')[0]
                # Loading for classification task
                image = Image.open(image_path)
                # the array based representation of the image will be used later in order to prepare the
                # result image with boxes and labels on it.
                image_np = load_image_into_numpy_array(image)
                original_img = image_np.copy()
                # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
                image_np_expanded = np.expand_dims(image_np, axis=0)
                
                if predicted_label_df.at[image_key,"Predicted_Class"] == 1:
                    # Actual detection.
                    (boxes, scores, classes, num) = sess.run(
                        [detection_boxes, detection_scores, detection_classes, num_detections],
                        feed_dict={image_tensor: image_np_expanded})
                    selected_indices = tf.image.non_max_suppression(
                                        boxes[0], scores[0], num[0], iou_threshold=iou_threshold,score_threshold=cutoff)
                    selected_boxes = tf.gather(boxes[0], selected_indices).eval()
                    selected_classes = tf.gather(classes[0], selected_indices).eval()
                    selected_scores = tf.gather(scores[0], selected_indices).eval()
                    
                    predicted_classes = list(selected_classes.astype(np.int32))
                    conf_scores = list(selected_scores)
                    print(list(zip(predicted_classes,conf_scores)))
                    if predicted_classes:
                        # Visualization of the results of a detection.
                        vis_util.visualize_boxes_and_labels_on_image_array(
                            image_np,
                            selected_boxes,
                            selected_classes.astype(np.int32),
                            selected_scores,
                            category_index,
                            use_normalized_coordinates=True,
                            line_thickness=8,
                            min_score_thresh=cutoff)
                        suffix = '_'.join(list(set([defect_cols[i-1].replace('/','_') for i in predicted_classes])))
                        if not os.path.exists('/home/ubuntu/dristi_ssd_model/iter7_20000/nms_cutoff0.4/test/known/'):
                            os.makedirs('/home/ubuntu/dristi_ssd_model/iter7_20000/nms_cutoff0.4/test/known/')
                        plt.imsave('/home/ubuntu/dristi_ssd_model/iter7_20000/nms_cutoff0.4/test/known/'+image_path.stem+'_'+suffix+'.jpg', image_np)
                        row = {"Image_key":image_key+'-'+suffix}
                        d = defaultdict(list)
                        [d[x[0]].append(x[1]) for x in zip([defect_cols[i-1] for i in predicted_classes],\
                                                           conf_scores)]
                        row.update(d)
                        image_summary = image_summary.append(row, ignore_index=True)
                    else:
                        print("unknown defect")
                        suffix = 'Unknown'
                        if not os.path.exists('/home/ubuntu/dristi_ssd_model/iter7_20000/nms_cutoff0.4/test/unknown/'):
                            os.makedirs('/home/ubuntu/dristi_ssd_model/iter7_20000/nms_cutoff0.4/test/unknown/')
                        plt.imsave('/home/ubuntu/dristi_ssd_model/iter7_20000/nms_cutoff0.4/test/unknown/'+image_path.stem+'_'+suffix+'.jpg', image_np)
                        row = {"Image_key":image_key+'-'+suffix}
                        d = defaultdict(list)
                        d['Unknown'].append(1-predicted_label_df.at[image_key,"Probability"])
                        row.update(d)
                        image_summary = image_summary.append(row, ignore_index=True)
#                     fig = plt.figure(figsize=IMAGE_SIZE)
#                     fig.add_subplot(1, 2, 1)
#                     plt.imshow(original_img)
#                     fig.add_subplot(1, 2, 2)
#                     plt.imshow(image_np)
#                     plt.show()
                    print("Time taken for object detection inference", time.time() - st_od)
                else:
                    print("no defect")
                    if not os.path.exists('/home/ubuntu/dristi_ssd_model/iter7_20000/nms_cutoff0.4/test/good/'):
                        os.makedirs('/home/ubuntu/dristi_ssd_model/iter7_20000/nms_cutoff0.4/test/good/')
                    suffix = 'Good'
                    plt.imsave('/home/ubuntu/dristi_ssd_model/iter7_20000/nms_cutoff0.4/test/good/'+image_path.stem+'_'+suffix+'.jpg', image_np)
                    row = {"Image_key":image_key+'-'+suffix}
                    d = defaultdict(list)
                    d['Good'].append(predicted_label_df.at[image_key,"Probability"])
                    row.update(d)
                    image_summary = image_summary.append(row, ignore_index=True)
                total_count+=1
            print("Defect Summary")
            def get_defect_conf_per_image(x):
                try:
                    l = np.mean(x)
                except:
                    l = None
                return l
            report1 = image_summary.set_index("Image_key")[defect_cols+['Unknown','Good']]\
                                    .applymap(get_defect_conf_per_image)
            print(report1)
            print("overall summary")
            overall_summary = pd.DataFrame(image_summary[defect_cols+['Unknown','Good']].count()\
                                           .append(pd.Series({'total':total_count})),columns = ['count'])
            overall_summary['Percentage'] = (overall_summary['count']*100)/total_count
            print(overall_summary)
            end = time.time()
            print("Time taken for overall object detection inference for all images",end - start)
            def get_defect_count_per_image(x):
                try:
                    l = len(x)
                except:
                    l = 0
                return l
            report2 = image_summary.set_index("Image_key")[defect_cols+['Unknown','Good']]\
                                    .applymap(get_defect_count_per_image)
            report2.to_csv("/home/ubuntu/dristi_ssd_model/iter7_20000/nms_cutoff0.4/test/report2_after_inference2.csv")

            
start_overall = time.time()
get_inference(binary_classification_inference(threshold,TEST_IMAGE_PATHS,model),TEST_IMAGE_PATHS,cutoff)
print("Total time taken for all images for complete inference",time.time()-start_overall)

Time taken for binary classification inference of 1 image 1.8561558723449707
Time taken for binary classification inference of 1 image 0.48079490661621094
Time taken for binary classification inference of 1 image 0.4716796875
Time taken for binary classification inference of 1 image 0.4804375171661377
Time taken for binary classification inference of 1 image 0.48114657402038574
Time taken for binary classification inference of 1 image 0.4770019054412842
Time taken for binary classification inference of 1 image 0.47886085510253906
Time taken for binary classification inference of 1 image 0.4809749126434326
Time taken for binary classification inference of 1 image 0.482637882232666
Time taken for binary classification inference of 1 image 0.4803035259246826
Time taken for binary classification inference of 1 image 0.47882843017578125
Time taken for binary classification inference of 1 image 0.47451186180114746
Time taken for binary classification inference of 1 image 0.48197031021118164


Time taken for binary classification inference of 1 image 0.48308658599853516
Time taken for binary classification inference of 1 image 0.4736976623535156
Time taken for binary classification inference of 1 image 0.47952961921691895
Time taken for binary classification inference of 1 image 0.479245662689209
Time taken for binary classification inference of 1 image 0.48206591606140137
Time taken for binary classification inference of 1 image 0.48144960403442383
Time taken for binary classification inference of 1 image 0.4848160743713379
Time taken for binary classification inference of 1 image 0.477785587310791
Time taken for binary classification inference of 1 image 0.4693727493286133
Time taken for binary classification inference of 1 image 0.46901702880859375
Time taken for binary classification inference of 1 image 0.47612667083740234
Time taken for binary classification inference of 1 image 0.47615981101989746
Time taken for binary classification inference of 1 image 0.47892856597

[(1, 0.64049214), (1, 0.52694947)]
Time taken for object detection inference 0.4251682758331299
/home/ubuntu/dristi_ssd_model/test_combined/VISUCI1_S1_20191228_042800_300_1.jpg
[(5, 0.40510094)]
Time taken for object detection inference 0.42400574684143066
/home/ubuntu/dristi_ssd_model/test_combined/VISUCI1_S1_20191228_042800_471_1.jpg
[(4, 0.5479232)]
Time taken for object detection inference 0.42224717140197754
/home/ubuntu/dristi_ssd_model/test_combined/VISUCI1_S1_20191228_042801_637_3.jpg
[(4, 0.69818866)]
Time taken for object detection inference 0.42241382598876953
/home/ubuntu/dristi_ssd_model/test_combined/VISUCI1_S1_20191228_042802_906_1.jpg
[(5, 0.77040017), (5, 0.41119182)]
Time taken for object detection inference 0.42896318435668945
/home/ubuntu/dristi_ssd_model/test_combined/VISUCI1_S1_20191228_042803_934_3.jpg
[(5, 0.7071084)]
Time taken for object detection inference 0.4262731075286865
/home/ubuntu/dristi_ssd_model/test_combined/VISUCI1_S1_20191228_042807_020_1.jpg
[]
u

[]
unknown defect
Time taken for object detection inference 0.4740254878997803
/home/ubuntu/dristi_ssd_model/test_combined/VISUCI1_S1_20200123_132947_702_3.jpg
[]
unknown defect
Time taken for object detection inference 0.47953033447265625
/home/ubuntu/dristi_ssd_model/test_combined/VISUCI1_S1_20200123_134650_411_3.jpg
[]
unknown defect
Time taken for object detection inference 0.4765968322753906
/home/ubuntu/dristi_ssd_model/test_combined/VISUCI1_S1_20200123_140452_982_3.jpg
[]
unknown defect
Time taken for object detection inference 0.48107147216796875
/home/ubuntu/dristi_ssd_model/test_combined/VISUCI1_S1_20200123_140553_188_3.jpg
[]
unknown defect
Time taken for object detection inference 0.48022937774658203
/home/ubuntu/dristi_ssd_model/test_combined/VISUCI1_S1_20200123_144741_485_3.jpg
[]
unknown defect
Time taken for object detection inference 0.4883108139038086
/home/ubuntu/dristi_ssd_model/test_combined/VISUCI1_S1_20200123_151254_391_3.jpg
[]
unknown defect
Time taken for obje

[(3, 0.99585986)]
Time taken for object detection inference 0.5535240173339844
/home/ubuntu/dristi_ssd_model/test_combined/VISUCI1_S1_20200328_215931_173_3.jpg
[(3, 0.95962846), (3, 0.9535489)]
Time taken for object detection inference 0.5539016723632812
/home/ubuntu/dristi_ssd_model/test_combined/VISUCI1_S1_20200329_132507_714_3.jpg
[(3, 0.67228717)]
Time taken for object detection inference 0.5605745315551758
/home/ubuntu/dristi_ssd_model/test_combined/VISUCI1_S1_20200405_230301_559_3.jpg
[]
unknown defect
Time taken for object detection inference 0.5473928451538086
/home/ubuntu/dristi_ssd_model/test_combined/VISUCI1_S1_20200408_055536_503_1.jpg
[]
unknown defect
Time taken for object detection inference 0.5498390197753906
/home/ubuntu/dristi_ssd_model/test_combined/VISUCI1_S1_20200410_074306_096_1.jpg
[]
unknown defect
Time taken for object detection inference 0.5511476993560791
/home/ubuntu/dristi_ssd_model/test_combined/VISUCI1_S1_20200410_074309_422_1.jpg
[(5, 0.5792536)]
Time ta

In [24]:
# os.remove("summary.csv") 

In [25]:
### ---end of code